<a href="https://colab.research.google.com/github/plue1011/code/blob/master/jit_mc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numba --upgrade

Requirement already up-to-date: numba in /usr/local/lib/python3.6/dist-packages (0.50.1)


In [1]:
import numba
numba.__version__

'0.50.1'

In [2]:
import networkx as nx
import numpy as np
import pandas as pd

from numba import njit
from numba.typed import Dict
from numba.typed import List
from numba.core import types

# データの読み込み

In [3]:
network = pd.read_csv('/content/drive/My Drive/fueki/Influence/data/WC.csv').values
g = network[:, :2].astype(np.int64)
w = network[:, 2]

In [4]:
# g = np.random.randint(0, 200, (1000, 2)).astype(np.float64)
# w = np.ones(1000).astype(np.float64) / 100
# network = np.hstack([g, w.reshape(-1, 1)])

# グラフの作成

In [5]:
int_array = types.int64[:]
int_tuple = numba.typeof((1e20,1e20))

@njit
def make_graph(network):
    g = network[:, :2].astype(np.int64)
    w = network[:, 2].astype(np.float64)

    # グラフ作成(エッジ)
    from_v = set(g[:, 0])
    G = Dict.empty(
        key_type=types.int64,
        value_type=int_array,
    )

    tmp = List()
    index_max = max(from_v)+1
    for v_index in range(index_max):
        tmp.append([v_index])

    for vu in g:
        tmp[vu[0]].append(vu[1])

    for index in tmp:
        G[index[0]] = np.array(index[1:], dtype=np.int64)

    # グラフ作成(確率)
    W = Dict.empty(
        key_type=int_tuple,
        value_type=types.float64,
    )

    for v, u, w in zip(network[:, 0], network[:, 1], network[:, 2]):
        W[(v, u)] = w

    return G, W

In [6]:
%time G, W = make_graph(network)

CPU times: user 2.84 s, sys: 69.1 ms, total: 2.91 s
Wall time: 2.91 s


In [7]:
G[2]

array([  5,  12,  18,  26,  30,  31,  33,  35,  48,  51,  58,  65,  72,
        77,  87,  89,  90,  93, 102, 103, 104, 105, 110, 123, 127, 155,
       233, 237, 297, 319, 331, 333, 335, 402, 544, 545, 547, 548, 549,
       550, 551, 552, 554, 555, 556, 557, 558, 559, 561, 562, 563, 564,
       565, 715])

# 幅優先探索

In [30]:
@njit
def IC_simulation(G, W, T):
    V = G.keys()
    maximum_inf = 0
    for v_i in V:
        expected_size = 0
        for t in range(T):
            visited = Dict.empty(
                key_type=types.int64,
                value_type=types.int64,
            )
            
            # bfs
            visited[v_i] = v_i
            queue = List([v_i])
            while queue:
                v = queue.pop(0)
                if v in G:
                    u_list = G[v]
                    for u in u_list:
                        if (W[(v, u)] > np.random.uniform(0, 1)) and (u not in visited):
                            queue.append(u)
                            visited[u] = v

            expected_size += len(visited)
        if expected_size / T > maximum_inf:
            maximum_node = v
            maximum_inf = expected_size / T
        
    return maximum_node

In [38]:
%time IC_simulation(G, W, 10)

1 loop, best of 3: 42.8 s per loop


In [33]:
int_array = types.int64[:]
int_tuple = numba.typeof((1e20,1e20))

@njit('float64(float64[:, :], int64)')
def expected_inf(network, T):
    g = network[:, :2].astype(np.int64)
    w = network[:, 2].astype(np.float64)

    # グラフ作成(エッジ)
    from_v = set(g[:, 0])
    G = Dict.empty(
        key_type=types.int64,
        value_type=int_array,
    )

    tmp = List()
    index_max = max(from_v)+1
    for v_index in range(index_max):
        tmp.append([v_index])

    for vu in g:
        tmp[vu[0]].append(vu[1])

    for index in tmp:
        G[index[0]] = np.array(index[1:], dtype=np.int64)

    # グラフ作成(確率)
    W = Dict.empty(
        key_type=int_tuple,
        value_type=types.float64,
    )

    for v, u, w in zip(network[:, 0], network[:, 1], network[:, 2]):
        W[(v, u)] = w

    # 影響力推定
    V = G.keys()
    maximum_inf = 0
    for v_i in V:
        expected_size = 0
        for t in range(T):
            visited = Dict.empty(
                key_type=types.int64,
                value_type=types.int64,
            )
            
            # bfs
            visited[v_i] = v_i
            queue = List([v_i])
            while queue:
                v = queue.pop(0)
                if v in G:
                    u_list = G[v]
                    for u in u_list:
                        if (W[(v, u)] > np.random.uniform(0, 1)) and (u not in visited):
                            queue.append(u)
                            visited[u] = v

            expected_size += len(visited)
        if expected_size / T > maximum_inf:
            maximum_node = v
            maximum_inf = expected_size / T
        
    return maximum_node

In [37]:
%time expected_inf(network, 50)

CPU times: user 43.8 s, sys: 57.1 ms, total: 43.8 s
Wall time: 43.6 s


35713.0